traffic signs tutorial: https://chsasank.github.io/keras-tutorial.html

# Description
---

- The problem I am tackling in this assignment is The German Traffic Sign Recognition Benchamark ([GTSRB](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news)).
- The German Traffic Sign Benchmark is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011. We cordially invite researchers from relevant fields to participate: The competition is designed to allow for participation without special domain knowledge. Our benchmark has the following properties:

    - Single-image, multi-class classification problem
    - More than 40 classes
    - More than 50,000 images in total
    - Large, lifelike database


## Tackling the problem and brainstorming
---

My approach to solve this problem will be a CNN as it's very successful and most commonly applied to analyzing visual imagery.  
- The tools will be used:
    - Keras
    - Convelutional Neural Network(CNN)
 


# Data Preprocessing
---

The dataset was donwloaded from the [GTSRB](https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/published-archive.html) website.

### Data normalization

- As I the images vary in size and color lets do [histogram equalization](https://en.wikipedia.org/wiki/Histogram_equalization) in `HSV` color space and resize the images to a standard size. **This method usually increases the global contrast of many images, especially when the usable data of the image is represented by close contrast values.**

- HSV = rgb2hsv( RGB ) converts the red, green, and blue values of an RGB image to hue, saturation, and value (HSV) values of an HSV image.

## Loading Data
---

In [1]:
import numpy as np
from skimage import io, color, exposure, transform
from sklearn.model_selection import train_test_split
import os
import glob
import h5py

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_first')

from matplotlib import pyplot as plt
%matplotlib inline

NUM_CLASSES = 43
IMG_SIZE = 48

Using TensorFlow backend.


## Image Preprocessing

In [2]:
# def preprocess_img(img):
#     # Histogram normalization in y
# #     hsv = color.rgb2hsv(img)
# #     hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
# #     img = color.hsv2rgb(hsv)
#     # central scrop
#     min_side = min(img.shape[:-1])
#     centre = img.shape[0]//2, img.shape[1]//2
#     img = img[centre[0]-min_side//2:centre[0]+min_side//2,
#               centre[1]-min_side//2:centre[1]+min_side//2,
#               :]
    
#     # rescale to standard size  
#     img = transform.resize(img, (IMG_SIZE, IMG_SIZE))
#     # roll color axis to axis 0
#     #img = np.rollaxis(img,-1)

#     return img


# def get_class(img_path):
#     return int(img_path.split('/')[-2])

def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])

## Preprocess all training images into a numpy array

In [3]:
try:
    with h5py.File('X.h5') as hf: 
        X, Y = hf['imgs'][:], hf['labels'][:]
    print("Loaded images from X.h5")
    
except (IOError,OSError, KeyError):  
    print("Error in reading X.h5. Processing all images...")
    root_dir = 'data/Final_Training/Images/'
    imgs = []
    labels = []

    all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
    np.random.shuffle(all_img_paths)
    for img_path in all_img_paths:
        try:
            img = preprocess_img(io.imread(img_path))
            label = get_class(img_path)
            imgs.append(img)
            labels.append(label)

            if len(imgs)%1000 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
        except (IOError, OSError):
            print('missed', img_path)
            pass

    X = np.array(imgs, dtype='float32')
    # Return a 2-D array with ones on the diagonal and zeros elsewhere
    Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

    with h5py.File('X.h5','w') as hf:
        hf.create_dataset('imgs', data=X)
        hf.create_dataset('labels', data=Y)

Loaded images from X.h5


In [14]:
# model compatible today's keras 
def cnn_model_1():
    
    INPUT_SHAPE = (3, 48, 48)
    model = Sequential()
    
    # 
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu', padding='same',
                     input_shape=INPUT_SHAPE))
    
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # 64   
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model


# old model from tutorial 
def cnn_model_2():
    INPUT_SHAPE = (3, 48, 48)
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same',
                     input_shape=(3, IMG_SIZE, IMG_SIZE),
                     activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model



In [18]:
Conv2D?

In [15]:
X.shape

(39209, 3, 48, 48)

# Models
- Let’s now define our models. We’ll use feed forward network with 6 convolutional layers followed by a fully connected hidden layer. We’ll also use dropout layers in between. Dropout regularizes the networks, i.e. it prevents the network from overfitting.

- All our layers have relu activations except the output layer. Output layer uses softmax activation as it has to output the probability for each of the classes.

- Sequential is a keras container for linear stack of layers. Each of the layers in the model needs to know the input shape it should expect, but it is enough to specify input_shape for the first layer of the Sequential model. Rest of the layers do automatic shape inference.

- To attach a fully connected layer (aka dense layer) to a convolutional layer, we will have to reshape/flatten the output of the conv layer. This is achieved by Flatten layer

- Go through the documentation of keras (relevant documentation : here and here) to understand what parameters for each of the layers mean.

## Compile the model

In [16]:
model = cnn_model_1()


# let's train the model using SGD + momentum (how original).
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)


model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])


def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))

In [17]:
batch_size = 32
nb_epoch = 5

model.fit(X, Y,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_split=0.2,
          shuffle=True,
          callbacks=[LearningRateScheduler(lr_schedule),
                    ModelCheckpoint('model.h5',save_best_only=True)]
            )

Train on 31367 samples, validate on 7842 samples
Epoch 1/5
31367/31367 [==============================] - 962s 31ms/step - loss: 1.7788 - acc: 0.4888 - val_loss: 0.3000 - val_acc: 0.9050
Epoch 2/5
31367/31367 [==============================] - 964s 31ms/step - loss: 0.3146 - acc: 0.9023 - val_loss: 0.1053 - val_acc: 0.9697
Epoch 3/5
31367/31367 [==============================] - 900s 29ms/step - loss: 0.1732 - acc: 0.9480 - val_loss: 0.0727 - val_acc: 0.9779
Epoch 4/5
31367/31367 [==============================] - 1721s 55ms/step - loss: 0.1247 - acc: 0.9623 - val_loss: 0.0523 - val_acc: 0.9861
Epoch 5/5
31367/31367 [==============================] - 945s 30ms/step - loss: 0.0941 - acc: 0.9716 - val_loss: 0.0337 - val_acc: 0.9901


In [5]:
# Load the test data

In [26]:
import pandas as pd
test = pd.read_csv('GT-final_test.csv',sep=';')

X_test = []
y_test = []
i = 0
for file_name, class_id  in zip(list(test['Filename']), list(test['ClassId'])):
    img_path = os.path.join('data/Final_Test/Images/',file_name)
    X_test.append(preprocess_img(io.imread(img_path)))
    y_test.append(class_id)
    
X_test = np.array(X_test)
y_test = np.array(y_test)




In [ ]:
## Model Evaluation 

In [27]:
y_pred = model.predict_classes(X_test)
acc = np.sum(y_pred==y_test)/np.size(y_pred)
print("Test accuracy = {}".format(acc))